In [1]:
from scraper import WebScraper
from time import sleep
import pandas as pd
from json import dumps
from tqdm import tqdm
from bs4 import BeautifulSoup
from glob import glob

In [2]:
cp = pd.read_excel('./cp_catalogue.xlsx', dtype=str)
cp.drop_duplicates(subset='Código Postal', inplace=True)
cp['Código Postal'] = cp['Código Postal'].str.zfill(5)
already_scraped = glob('./results/rappi_new/*')
already_scraped = set(a.split('/')[-1].split('.')[0].split('_')[-1] for a in already_scraped)
cp = cp[~cp['Código Postal'].isin(already_scraped)]

direcciones = [f'{c}, Ciudad de México' for c in cp['Código Postal']]

In [3]:
len(pd.read_excel('./cp_catalogue.xlsx', dtype=str)['Código Postal'].unique())

1110

In [4]:
source_cates = dict(
    populares_cerca = "https://www.rappi.com.mx/"
    )

source_cate = "populares_cerca"

In [5]:
for current_address in tqdm(direcciones):
    
    scraper = WebScraper()

    scraper.navigate_to_url(source_cates["populares_cerca"])

    scraper.click_element('//*[@id="rappi-web-toolbar"]/div[2]/div/div/div/span')

    final_address = scraper.enter_address(current_address,
                                        input_xpath="//div[starts-with(@id, 'chakra-modal--body-')]//input",
                                        selection_xpath="//div[starts-with(@id, 'chakra-modal--body-')]//ul/li[1]/button"
                                        )

    # Confirm address
    try:
        scraper.click_element('//*[@id="confirm-address-button"]')

        sleep(2)
        # Save address
        scraper.click_element('//*[@id="save-address-button"]')
    except:
        scraper.close()
        
        with open(f'./results/rappi_new/error_{current_address.split(',')[0]}.txt', 'w') as f:
            f.write('Error al seleccionar dirección (No data)\n')
            
        continue

    cards_data = scraper.scrape_cards(
        cards_xpath='//*[@id="__next"]/div[3]/div[2]/div/div[2]/div',
        aditional_info={"cate_source": source_cate,
                        "current_address": current_address,
                        "final_address": final_address,
                        "platform": 'rappi'
                        }
    )
    
    
    with open(f'./results/rappi_new/rappi_new_{source_cate}_{current_address.split(',')[0]}.json', 'w') as f:
        f.write(dumps(cards_data, indent=4))
    
    soup = BeautifulSoup(scraper.driver.page_source, 'html.parser')
    with open(f'./results/rappi_new/rappi_new_{source_cate}_{current_address.split(',')[0]}.html', 'w') as f:
        f.write(soup.prettify()) 
    
    
    scraper.close()
    
    sleep(1)

  0%|          | 0/1110 [00:00<?, ?it/s]

2025-11-14 08:38:30,265 - INFO - Navegando a la URL: https://www.rappi.com.mx/
2025-11-14 08:38:34,526 - INFO - Elemento en //*[@id="rappi-web-toolbar"]/div[2]/div/div/div/span clickeado correctamente.
2025-11-14 08:38:37,758 - INFO - Elemento en //div[starts-with(@id, 'chakra-modal--body-')]//ul/li[1]/button clickeado correctamente.
2025-11-14 08:38:37,759 - INFO - Dirección ingresada correctamente.
2025-11-14 08:38:47,963 - ERROR - No se pudo hacer clic en el elemento en //*[@id="confirm-address-button"]: Message: 
Stacktrace:
#0 0x56ccbb5dab9a <unknown>
#1 0x56ccbb2c0670 <unknown>
#2 0x56ccbb30fc48 <unknown>
#3 0x56ccbb30fee1 <unknown>
#4 0x56ccbb356924 <unknown>
#5 0x56ccbb334bad <unknown>
#6 0x56ccbb353dd6 <unknown>
#7 0x56ccbb334923 <unknown>
#8 0x56ccbb3026e7 <unknown>
#9 0x56ccbb3036de <unknown>
#10 0x56ccbb5a466b <unknown>
#11 0x56ccbb5a8611 <unknown>
#12 0x56ccbb5904e5 <unknown>
#13 0x56ccbb5a9192 <unknown>
#14 0x56ccbb5756ef <unknown>
#15 0x56ccbb5c99d8 <unknown>
#16 0x56ccb